# Introduction
To do downstream analysis later in the project we need:
1. Geolocation of where the sample was taken
2. Source that the sample was isolated from
3. Date on which the sample was collected
4. Filter out lab strains

None of these things have a column in any of the tables from the NCBI database. See this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6380228/) that describes how metadata in NCBI sucks.
 
However, there is a column called 'sample_attribute' in the SRA and Sample table where a submitter can add additional information about a sample. As 'sample_attribute' does not require a specific format or specific information. The information found there varies greatly between samples. Some organizations (rivm) that submit data to the NCBI have a consisted format for this column which then also varies per organization, others do not. This makes it very challenging to extrapolate the information mentioned above for all samples. In this notebook we attempt to extract this information.

In [ ]:
import numpy as np
import pandas as pd
import pyarrow.feather as feather
from collections import defaultdict

Functions written for this notebook are stored in wrangling_funcs.py. Please look there for documentation and tests.

In [ ]:
import wrangling_funcs

# Reading in the data
---

R has a nice package called SRAdb that you can use to query the NCBI database. However, I prefer working in Python. So we are querying the data in R using SRAdb and then exporting it in feather format for use here. There might be a way to directly get a dump of the SRA database and query it without using SRAdb. I will look into this.

The default index of a dataframe is not useful to us. Instead, we use the run_accession, these should be unique. This way we can keep track when we split the metadata into a separate dataframe.

In [ ]:
# data = feather.read_feather(file_path)
data = feather.read_feather(snakemake.input[0])
df = pd.DataFrame(data)
print(f'Is run_accession unique?: {df["run_accession"].is_unique}')
# print(df.shape)
df.set_index('run_accession', inplace=True)

print(f'---Number of rows: {df.shape[0]}, Number of columns: {df.shape[1]}---')
df.head()

# Finding metadata in the sample_attribute
---

All the metadata we are interested in is contained in the 'sample_attribute' column. From what we could see most of the information in this column is split by '||' characters. The information between these characters is then often split using ':'. We will use this to make key value pairs which we will then turn into a dataframe.

In [ ]:
metadata = df['sample_attribute']
faulty_lines = []
meta_lines = []

for line, identity in zip(metadata, metadata.index):
    line = line.split("||")
    line_items = defaultdict(list)
    for subitem in line:
        try:
            key, value = subitem.split(': ', 1)
            strip_key = wrangling_funcs.clean_string(key)
            strip_value = value.strip()
            line_items[strip_key] = strip_value
            line_items['run_accession'] = identity
        except ValueError:
            faulty_lines.append((identity, line))
    meta_lines.append(line_items)

meta_df = pd.DataFrame(meta_lines)
meta_df.set_index('run_accession', inplace=True)

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

### Replacing missing values with np.nan
Many of the entries in the database contain words such as 'missing', 'not available', 'not applicable' instead of NaN values. Here we replace those values by NaN's. Key word selection was done based on what we saw when looking through the dataframe.

In [ ]:
replace_dict = {'^\*$': np.nan, '^-$': np.nan,
                '^\.$': np.nan, '^[Nn]one$': np.nan,
                '^[Nn]an$': np.nan, '^[Uu]nknown$': np.nan,
                '(?i)^not[ _-]collected$': np.nan, '(?i)^not[ _-]provided': np.nan,
                '^\?$': np.nan, '^ $': np.nan,
                '(?i)^not[ _-]applicable$': np.nan, '^[Nn]a$': np.nan,
                '^[Nn]o$': np.nan, '^[Oo]ther$': np.nan,
                '^[Mm]is{1,3}ing$': np.nan, '^[Uu]nspecified$': np.nan,
                '^[Nn]ot[ ]available$': np.nan, '^[Nn]ot[ :]available[:] not collected$': np.nan,
                '^[Nn]ot[ :]available[:] to be reported later$': np.nan}

meta_df = meta_df.replace(to_replace=replace_dict, regex=True)

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

### Searching for geographic data

There is no consistent column that contains the geolocation. To (hopefully) obtain the geolocation we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.

In [ ]:
geo_col_matches = wrangling_funcs.find_columns(['geo', 'geographic', 'country', 'continent'], meta_df, ['longitude', 'latitude', 'depth'])
print(f'The following columns matched the keywords: {geo_col_matches}')
meta_df = wrangling_funcs.combine_columns(meta_df, list(geo_col_matches), 'inferred_location')
# meta_df.drop(geo_col_matches, inplace=True, axis=1)

meta_df['inferred_continent'], meta_df['inferred_country'], meta_df['inferred_city'] = zip(*meta_df['inferred_location'].map(wrangling_funcs.clean_geo))
meta_df.drop('inferred_location', axis=1, inplace=True)

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

### Searching for the sample collection data

There is no consistent column that contains the date. To (hopefully) obtain the date we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.

In [ ]:
date_col_matches = wrangling_funcs.find_columns(['date', 'year'], meta_df, ['update'])
print(f'The following columns matched the keywords: {date_col_matches}')
meta_df = wrangling_funcs.combine_columns(meta_df, list(date_col_matches), 'inferred_collection_year')
# meta_df.drop(date_col_matches, inplace=True, axis=1)


date = meta_df['inferred_collection_year'].str.extract(r'^(\d{4})', expand=False) # Extract the year
meta_df['inferred_collection_year'] = pd.to_numeric(date) # cast year to int
# meta_df['inferred_collection_year']

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

### Searching for sample isolation source
There is no consistent column that contains the isolation source. To (hopefully) obtain the isolation source we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.


In [ ]:
isolate_matches = wrangling_funcs.find_columns(['sample', 'source', 'environment', 'env', 'site'], meta_df, ['name', 'provider', 'comment'])
print(isolate_matches)

meta_df = wrangling_funcs.combine_columns(meta_df, list(isolate_matches), "inferred_source")
# meta_df.drop(isolate_matches, inplace=True, axis=1)

meta_df['inferred_source'] = meta_df['inferred_source'].apply(wrangling_funcs.clean_source)

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

### First round removal of lab strains
We don't want sample from lab strains, so we filter out rows based on some keywords

In [ ]:
keywords_to_exclude = ["replicate", "mutant"]
meta_df = meta_df[~meta_df.sample_comment.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
meta_df = meta_df[~meta_df.genotype.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
meta_df = meta_df[~meta_df.lab_experiment_type.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

### Remove non relevant columns
We have a ton of columns and very few of them are actually usefull to us. Let's remove all not relevant columns

In [ ]:
meta_df = meta_df[['strain','inferred_collection_year','inferred_source','inferred_continent', 'inferred_country', 'inferred_city','geographic_location_latitude', 'geographic_location_longitude']]

print(f'---Number of rows: {meta_df.shape[0]}, Number of columns: {meta_df.shape[1]}---')
meta_df.head()

# Combine sample_attribute metadata with rest of the data
---
Now that we have extracted the metadata that we wanted we can combine it back to the original dataframe. We only want to keep rows that have values for the collection_year/source/country because we require this downstream.

In [ ]:
combined_df = df.join(meta_df)
cols = ['inferred_collection_year', 'inferred_source', 'inferred_country']
combined_df = combined_df.dropna(subset=cols)

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Remove lab strain samples from combined dataframe
After combining the dataframes we have new columns that might give us more information if a particular run comes from a lab strain, so we filter again.


In [ ]:
keywords_to_exclude = ["replicate", "mutant"]

combined_df = combined_df[~combined_df.description.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_title.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_abstract.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_description.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Remove samples submitted by the RIVM
Since we will already have the RIVM samples in the database there is no need to have them here.

In [ ]:
combined_df = combined_df[~combined_df.submission_center.str.contains("RIVM", na=False)] # Remove samples submitted by the RIVM

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Replace missing values with np.nan
Once again after combining and obtaining new columns there are values in the rows that indicate NaNs

In [ ]:
replace_dict = {'^\*$': np.nan, '^-$': np.nan,
                '^\.$': np.nan, '^[Nn]one$': np.nan,
                '^[Nn]an$': np.nan, '^[Uu]nknown$': np.nan,
                '(?i)^not[ _-]collected$': np.nan, '(?i)^not[ _-]provided': np.nan,
                '^\?$': np.nan, '^ $': np.nan,
                '(?i)^not[ _-]applicable$': np.nan, '^[Nn]a$': np.nan,
                '^[Nn]o$': np.nan, '^[Oo]ther$': np.nan,
                '^[Mm]is{1,3}ing$': np.nan, '^[Uu]nspecified$': np.nan,
                '^[Nn]ot[ ]available$': np.nan, '^[Nn]ot[ :]available[:] not collected$': np.nan,
                '^[Nn]ot[ :]available[:] to be reported later$': np.nan}

combined_df = combined_df.replace(to_replace=replace_dict, regex=True)

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Throw away empty columns
We want to filter out columns that only have NaN values so there is less cluter

In [ ]:
combined_df = combined_df.dropna(axis=1, how='all')

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Split data on platform
Lastly we would like to split up the runs based on the run platform since they will require different software to analyse 

In [ ]:
platforms = [platform for _, platform in combined_df.groupby(['platform'])]

illumina_df = platforms[0]
nanopore_df = platforms[1]
pacbio_df = platforms[2]

## ILLUMINA

In [ ]:
print(f'---Number of rows: {illumina_df.shape[0]}, Number of columns: {illumina_df.shape[1]}---')
illumina_df.head()

In [ ]:
illumina_df[['spots', 'bases', 'inferred_collection_year']].describe()

In [ ]:
illumina_df['scientific_name'].value_counts()

In [ ]:
illumina_df['inferred_collection_year'].value_counts()

## OXFORD_NANOPORE 

In [ ]:
print(f'---Number of rows: {nanopore_df.shape[0]}, Number of columns: {nanopore_df.shape[1]}---')
nanopore_df.head()

In [ ]:
nanopore_df[['spots', 'bases', 'inferred_collection_year']].describe()

In [ ]:
nanopore_df['scientific_name'].value_counts()

In [ ]:
nanopore_df['inferred_collection_year'].value_counts()

## PACBIO_SMRT

In [ ]:
print(f'---Number of rows: {pacbio_df.shape[0]}, Number of columns: {pacbio_df.shape[1]}---')
pacbio_df.head()

In [ ]:
pacbio_df[['spots', 'bases', 'inferred_collection_year']].describe()

In [ ]:
pacbio_df['scientific_name'].value_counts()

In [ ]:
pacbio_df['inferred_collection_year'].value_counts()

## Write out files

In [ ]:
# illumina_df.to_csv('../results/ILLUMINA.csv')
# nanopore_df.to_csv('../results/OXFORD_NANOPORE.csv')
# pacbio_df.to_csv('../results/PACBIO_SMRT.csv')

illumina_df.to_csv(snakemake.output[0], sep='\t', na_rep='NaN')
nanopore_df.to_csv(snakemake.output[1], sep='\t', na_rep='NaN')
pacbio_df.to_csv(snakemake.output[2], sep='\t', na_rep='NaN')
